## WeGo Route 50 Data Time-Based Contextual Segmentation

In [1]:
import pandas as pd
import numpy as np
from pandas.tseries.holiday import USFederalHolidayCalendar

In [2]:
wego = pd.read_csv("../data/Route 50 Timepoint and Headway Data, 1-1-2023 through 5-12-2025.csv")

In [3]:
wego.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 618998 entries, 0 to 618997
Data columns (total 30 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   CALENDAR_ID                618998 non-null  int64  
 1   SERVICE_ABBR               618998 non-null  int64  
 2   ADHERENCE_ID               618998 non-null  int64  
 3   DATE                       618998 non-null  object 
 4   ROUTE_ABBR                 618998 non-null  int64  
 5   BLOCK_ABBR                 618998 non-null  int64  
 6   OPERATOR                   618998 non-null  int64  
 7   TRIP_ID                    618998 non-null  int64  
 8   OVERLOAD_ID                618998 non-null  int64  
 9   ROUTE_DIRECTION_NAME       618998 non-null  object 
 10  TIME_POINT_ABBR            618998 non-null  object 
 11  ROUTE_STOP_SEQUENCE        618998 non-null  int64  
 12  TRIP_EDGE                  618998 non-null  int64  
 13  LATITUDE                   61

### 1. Clean time columns (SCHEDULED, ARRIVAL, DEPARTURE)
Some time columns contain "1900-01-01" as a prefix (Excel default behavior).
We strip that out and convert values to Python `datetime.time` format.


In [4]:
wego['DATE'] = pd.to_datetime(wego['DATE'], errors='coerce')

wego['SCHEDULED_TIME'] = wego['SCHEDULED_TIME'].astype(str)
wego['SCHEDULED_TIME'] = wego['SCHEDULED_TIME'].str.replace("1900-01-01 ", "", regex=False)
wego['SCHEDULED_TIME'] = wego['SCHEDULED_TIME'].str.strip()

wego['SCHEDULED_TIME'] = pd.to_datetime(wego['SCHEDULED_TIME'], format='%H:%M:%S', errors='coerce').dt.time

In [5]:
wego['ACTUAL_ARRIVAL_TIME'] = wego['ACTUAL_ARRIVAL_TIME'].astype(str)
wego['ACTUAL_ARRIVAL_TIME'] = wego['ACTUAL_ARRIVAL_TIME'].str.replace("1900-01-01 ", "", regex=False)
wego['ACTUAL_ARRIVAL_TIME'] = wego['ACTUAL_ARRIVAL_TIME'].str.strip()

wego['ACTUAL_ARRIVAL_TIME'] = pd.to_datetime(wego['ACTUAL_ARRIVAL_TIME'], format='%H:%M:%S', errors='coerce').dt.time

In [6]:
wego['ACTUAL_DEPARTURE_TIME'] = wego['ACTUAL_DEPARTURE_TIME'].astype(str)
wego['ACTUAL_DEPARTURE_TIME'] = wego['ACTUAL_DEPARTURE_TIME'].str.replace("1900-01-01 ", "", regex=False)
wego['ACTUAL_DEPARTURE_TIME'] = wego['ACTUAL_DEPARTURE_TIME'].str.strip()

wego['ACTUAL_DEPARTURE_TIME'] = pd.to_datetime(wego['ACTUAL_DEPARTURE_TIME'], format='%H:%M:%S', errors='coerce').dt.time

### 2. Feature Engineering – Time-Based Segmentation
We add the following contextual columns:
- `TIME_OF_DAY`: AM Peak, PM Peak, etc.
- `SEASON`: Winter, Spring, etc.
- `IS_HOLIDAY`: US federal holiday flag
- `IN_SCHOOL_ZONE_TIME`: True if trip occurs during school zone speed restriction hours
- `DAY_TYPE`: Weekday/Saturday/Sunday

In [7]:
# 1. TIME_OF_DAY
t_6am = pd.to_datetime("06:00").time()
t_9am = pd.to_datetime("09:00").time()
t_3pm = pd.to_datetime("15:00").time()
t_7pm = pd.to_datetime("19:00").time()
t_11pm = pd.to_datetime("23:00").time()

def get_time_of_day(t):
    if pd.isna(t): 
        return None
    if t_6am <= t < t_9am:
        return 'AM Peak'
    elif t < t_3pm:
        return 'Midday'
    elif t < t_7pm:
        return 'PM Peak'
    elif t < t_11pm:
        return 'Evening'
    else:
        return 'Overnight'

wego['TIME_OF_DAY'] = wego['SCHEDULED_TIME'].apply(get_time_of_day)

In [8]:
# 2. SEASON
def get_season(d):
    if pd.isna(d): 
        return None
    m = d.month
    return ('Winter' if m in [12,1,2] else
            'Spring' if m in [3,4,5] else
            'Summer' if m in [6,7,8] else
            'Fall')

wego['SEASON'] = wego['DATE'].apply(get_season)

In [9]:
# 3. IS_HOLIDAY
cal = USFederalHolidayCalendar()
holidays = cal.holidays(start=wego['DATE'].min(), end=wego['DATE'].max())

wego['IS_HOLIDAY'] = wego['DATE'].isin(holidays)

In [10]:
# 4. IN_SCHOOL_ZONE_TIME
is_not_summer = ~wego['DATE'].dt.month.isin([6, 7])

# (SERVICE_ABBR == 1 → Weekday)
is_weekday = wego['SERVICE_ABBR'] == 1

is_not_holiday = wego['IS_HOLIDAY'] == False

wego['SCHEDULED_MINUTES'] = wego['SCHEDULED_TIME'].apply(lambda x: x.hour * 60 + x.minute if pd.notna(x) else np.nan)

# School time zone:
in_morning_zone = (wego['SCHEDULED_MINUTES'] >= 420) & (wego['SCHEDULED_MINUTES'] < 540)      # 07:00–09:00
in_afternoon_zone = (wego['SCHEDULED_MINUTES'] >= 870) & (wego['SCHEDULED_MINUTES'] < 990)    # 14:30–16:30

wego['IN_SCHOOL_ZONE_TIME'] = np.where(
    is_not_summer & is_weekday & is_not_holiday & (in_morning_zone | in_afternoon_zone),
    'Y', 'N'
)

In [11]:
# 5. DAY_TYPE
wego['DAY_TYPE'] = wego['SERVICE_ABBR'].map({1: 'Weekday', 2: 'Saturday', 3: 'Sunday'})

### 3. Save the cleaned and enriched dataset

In [12]:
wego.to_csv("../data/wego_contextual_segmentation.csv", index=False)

In [13]:
wego

,CALENDAR_ID,SERVICE_ABBR,ADHERENCE_ID,DATE,ROUTE_ABBR,BLOCK_ABBR,OPERATOR,TRIP_ID,OVERLOAD_ID,ROUTE_DIRECTION_NAME,...,PREV_SCHED_STOP_CANCELLED,IS_RELIEF,BLOCK_STOP_ORDER,DWELL_IN_MINS,TIME_OF_DAY,SEASON,IS_HOLIDAY,SCHEDULED_MINUTES,IN_SCHOOL_ZONE_TIME,DAY_TYPE
0,120230101,3,93549161,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,0.0,0,2,8.133333,Midday,Winter,False,334,N,Sunday
1,120230101,3,93549162,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,0.0,0,5,0.000000,Midday,Winter,False,340,N,Sunday
2,120230101,3,93549163,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,0.0,0,11,0.000000,Midday,Winter,False,347,N,Sunday
3,120230101,3,93549164,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,0.0,0,13,0.000000,Midday,Winter,False,350,N,Sunday
4,120230101,3,93549165,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,0.0,0,18,2.150000,Midday,Winter,False,354,N,Sunday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618993,120250512,1,119531049,2025-05-12,50,8401,3077,432121,0,FROM DOWNTOWN,...,NaN,0,23,4.500000,AM Peak,Spring,False,460,Y,Weekday
618994,120250512,1,119531638,2025-05-12,50,9302,3246,432353,0,TO DOWNTOWN,...,0.0,0,2,27.283333,PM Peak,Spring,False,905,Y,Weekday
618995,120250512,1,119531639,2025-05-12,50,9302,3246,432353,0,TO DOWNTOWN,...,NaN,0,3,0.133333,PM Peak,Spring,False,918,Y,Weekday
618996,120250512,1,119531770,2025-05-12,50,9950,2448,432387,0,TO DOWNTOWN,...,0.0,0,2,8.166666,PM Peak,Spring,False,920,Y,Weekday


In [14]:
cln_wego = wego[(wego['OVERLOAD_ID'] == 0) &
(wego['TRIP_EDGE'] == 0) &
(wego['STOP_CANCELLED'] == 0) &
(~wego['TIME_POINT_ABBR'].str.contains('HLWD', na=False))]

In [15]:
cln_wego

,CALENDAR_ID,SERVICE_ABBR,ADHERENCE_ID,DATE,ROUTE_ABBR,BLOCK_ABBR,OPERATOR,TRIP_ID,OVERLOAD_ID,ROUTE_DIRECTION_NAME,...,PREV_SCHED_STOP_CANCELLED,IS_RELIEF,BLOCK_STOP_ORDER,DWELL_IN_MINS,TIME_OF_DAY,SEASON,IS_HOLIDAY,SCHEDULED_MINUTES,IN_SCHOOL_ZONE_TIME,DAY_TYPE
2,120230101,3,93549163,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,0.0,0,11,0.00,Midday,Winter,False,347,N,Sunday
3,120230101,3,93549164,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,0.0,0,13,0.00,Midday,Winter,False,350,N,Sunday
4,120230101,3,93549165,2023-01-01,50,5000,2355,332422,0,TO DOWNTOWN,...,0.0,0,18,2.15,Midday,Winter,False,354,N,Sunday
7,120230101,3,93549168,2023-01-01,50,5000,2355,332423,0,FROM DOWNTOWN,...,0.0,0,33,0.00,AM Peak,Winter,False,385,N,Sunday
8,120230101,3,93549169,2023-01-01,50,5000,2355,332423,0,FROM DOWNTOWN,...,0.0,0,38,0.00,AM Peak,Winter,False,389,N,Sunday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618979,120250512,1,119524761,2025-05-12,50,5006,754,429696,0,TO DOWNTOWN,...,0.0,0,67,0.00,PM Peak,Spring,False,1004,N,Weekday
618980,120250512,1,119524762,2025-05-12,50,5006,754,429696,0,TO DOWNTOWN,...,0.0,0,72,0.00,PM Peak,Spring,False,1008,N,Weekday
618983,120250512,1,119524765,2025-05-12,50,5006,754,429697,0,FROM DOWNTOWN,...,0.0,0,87,0.00,PM Peak,Spring,False,1047,N,Weekday
618984,120250512,1,119524766,2025-05-12,50,5006,754,429697,0,FROM DOWNTOWN,...,0.0,0,92,0.00,PM Peak,Spring,False,1052,N,Weekday


In [16]:
non_nan_values = cln_wego['SCHEDULED_HDWY'].dropna()

In [17]:
non_null_headway = cln_wego[['DATE', 'DAY_TYPE', 'TIME_OF_DAY', 'SEASON', 'IN_SCHOOL_ZONE_TIME', 'IS_HOLIDAY', 'OPERATOR', 'TIME_POINT_ABBR', 'ROUTE_DIRECTION_NAME', 'SCHEDULED_TIME', 'ACTUAL_ARRIVAL_TIME', 'ADHERENCE', 'DWELL_IN_MINS', 'SCHEDULED_HDWY', 'ACTUAL_HDWY', 'HDWY_DEV']].dropna()
non_null_headway 

,DATE,DAY_TYPE,TIME_OF_DAY,SEASON,IN_SCHOOL_ZONE_TIME,IS_HOLIDAY,OPERATOR,TIME_POINT_ABBR,ROUTE_DIRECTION_NAME,SCHEDULED_TIME,ACTUAL_ARRIVAL_TIME,ADHERENCE,DWELL_IN_MINS,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV
14,2023-01-01,Sunday,AM Peak,Winter,N,False,2355,WHBG,TO DOWNTOWN,07:17:00,07:20:48,-5.483333,1.683333,30.0,34.016666,4.016666
15,2023-01-01,Sunday,AM Peak,Winter,N,False,2355,CH46,TO DOWNTOWN,07:20:00,07:24:18,-4.300000,0.000000,30.0,34.066666,4.066666
16,2023-01-01,Sunday,AM Peak,Winter,N,False,2355,28&CHARL,TO DOWNTOWN,07:24:00,07:26:41,-2.683333,0.000000,30.0,32.516666,2.516666
19,2023-01-01,Sunday,AM Peak,Winter,N,False,2355,28&CHARL,FROM DOWNTOWN,07:55:00,07:56:33,-1.550000,0.000000,30.0,31.233333,1.233333
20,2023-01-01,Sunday,AM Peak,Winter,N,False,2355,CH46,FROM DOWNTOWN,07:59:00,07:59:04,-2.533333,2.466666,30.0,32.166666,2.166666
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
618979,2025-05-12,Weekday,PM Peak,Spring,N,False,754,CH46,TO DOWNTOWN,16:44:00,16:49:25,-5.416666,0.000000,15.0,15.900000,0.900000
618980,2025-05-12,Weekday,PM Peak,Spring,N,False,754,28&CHARL,TO DOWNTOWN,16:48:00,16:53:47,-5.783333,0.000000,15.0,15.983333,0.983333
618983,2025-05-12,Weekday,PM Peak,Spring,N,False,754,28&CHARL,FROM DOWNTOWN,17:27:00,17:28:46,-1.766666,0.000000,15.0,11.383333,-3.616667
618984,2025-05-12,Weekday,PM Peak,Spring,N,False,754,CH46,FROM DOWNTOWN,17:32:00,17:33:19,-1.316666,0.000000,15.0,9.816666,-5.183334


In [18]:
non_null_headway['HDWY_DEV_PCT'] = (non_null_headway['HDWY_DEV'] / non_null_headway['SCHEDULED_HDWY']) * 100
non_null_headway.sort_values(['HDWY_DEV'])

,DATE,DAY_TYPE,TIME_OF_DAY,SEASON,IN_SCHOOL_ZONE_TIME,IS_HOLIDAY,OPERATOR,TIME_POINT_ABBR,ROUTE_DIRECTION_NAME,SCHEDULED_TIME,ACTUAL_ARRIVAL_TIME,ADHERENCE,DWELL_IN_MINS,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,HDWY_DEV_PCT
155958,2023-08-10,Weekday,AM Peak,Summer,N,False,1266,WHBG,FROM DOWNTOWN,06:32:00,06:54:56,-22.933333,0.000000,35.0,1.750000,-33.250000,-95.000000
354702,2024-05-11,Saturday,Overnight,Spring,N,False,1742,28&CHARL,TO DOWNTOWN,23:54:00,23:53:08,-1.066666,1.933333,30.0,0.016666,-29.983334,-99.944447
354263,2024-05-11,Saturday,Evening,Spring,N,False,3102,WHBG,FROM DOWNTOWN,22:01:00,22:06:28,-7.866666,2.400000,30.0,0.233333,-29.766667,-99.222223
134741,2023-07-11,Weekday,Evening,Summer,N,False,1623,WHBG,TO DOWNTOWN,19:47:00,20:21:16,-34.266666,0.000000,31.0,1.733333,-29.266667,-94.408603
155957,2023-08-10,Weekday,AM Peak,Summer,N,False,1266,CH46,FROM DOWNTOWN,06:28:00,06:35:30,-25.083333,17.583333,35.0,5.816666,-29.183334,-83.380954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284054,2024-02-05,Weekday,Evening,Winter,N,False,304,CH46,TO DOWNTOWN,19:20:00,19:45:57,-25.950000,0.000000,30.0,129.366666,99.366666,331.222220
284053,2024-02-05,Weekday,Evening,Winter,N,False,304,WHBG,TO DOWNTOWN,19:16:00,19:41:02,-26.900000,1.866666,30.0,130.233333,100.233333,334.111110
269020,2024-01-15,Saturday,Evening,Winter,N,True,907,WHBG,TO DOWNTOWN,19:47:00,19:47:04,-0.066666,0.000000,30.0,139.300000,109.300000,364.333333
71119,2023-04-11,Weekday,Overnight,Spring,N,False,304,28&CHARL,TO DOWNTOWN,23:54:00,23:54:13,-0.216666,0.000000,30.0,146.500000,116.500000,388.333333


In [19]:
non_null_headway.sort_values(['HDWY_DEV_PCT'])

,DATE,DAY_TYPE,TIME_OF_DAY,SEASON,IN_SCHOOL_ZONE_TIME,IS_HOLIDAY,OPERATOR,TIME_POINT_ABBR,ROUTE_DIRECTION_NAME,SCHEDULED_TIME,ACTUAL_ARRIVAL_TIME,ADHERENCE,DWELL_IN_MINS,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,HDWY_DEV_PCT
90619,2023-05-09,Weekday,Midday,Spring,Y,False,1198,28&CHARL,FROM DOWNTOWN,14:42:00,14:44:02,-2.033333,0.000000,16.0,0.000000,-16.000000,-100.000000
142738,2023-07-22,Saturday,Midday,Summer,N,False,2876,CH46,FROM DOWNTOWN,12:09:00,12:11:35,-2.583333,0.000000,20.0,0.000000,-20.000000,-100.000000
524475,2025-01-02,Weekday,PM Peak,Winter,Y,False,357,28&CHARL,TO DOWNTOWN,15:48:00,15:45:33,2.450000,0.000000,14.0,0.000000,-14.000000,-100.000000
13044,2023-01-19,Weekday,PM Peak,Winter,N,False,2109,CH46,TO DOWNTOWN,16:50:00,16:53:36,-3.600000,0.000000,15.0,0.000000,-15.000000,-100.000000
65974,2023-04-04,Weekday,PM Peak,Spring,N,False,1927,28&CHARL,FROM DOWNTOWN,17:42:00,17:42:18,-0.300000,0.000000,15.0,0.000000,-15.000000,-100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71119,2023-04-11,Weekday,Overnight,Spring,N,False,304,28&CHARL,TO DOWNTOWN,23:54:00,23:54:13,-0.216666,0.000000,30.0,146.500000,116.500000,388.333333
529637,2025-01-09,Weekday,PM Peak,Winter,Y,False,2033,CH46,TO DOWNTOWN,16:29:00,17:14:42,-47.566666,1.866666,15.0,73.633333,58.633333,390.888887
529638,2025-01-09,Weekday,PM Peak,Winter,N,False,2033,28&CHARL,TO DOWNTOWN,16:33:00,17:20:39,-49.350000,1.700000,15.0,75.183333,60.183333,401.222220
529636,2025-01-09,Weekday,PM Peak,Winter,Y,False,2033,WHBG,TO DOWNTOWN,16:25:00,17:13:12,-48.200000,0.000000,15.0,76.966666,61.966666,413.111107


In [23]:
hdwy_dev_filtered= non_null_headway[non_null_headway['HDWY_DEV'].between(15, 30, inclusive='both')]
# hdwy_dev_filtered 

In [26]:
hdwy_per_filtered= non_null_headway[non_null_headway['HDWY_DEV_PCT'].between(150, 500, inclusive='both')]
# hdwy_per_filtered

In [34]:
hdwy_dev_filtered[(hdwy_dev_filtered['ADHERENCE'] < -20)].sort_values('ADHERENCE')

,DATE,DAY_TYPE,TIME_OF_DAY,SEASON,IN_SCHOOL_ZONE_TIME,IS_HOLIDAY,OPERATOR,TIME_POINT_ABBR,ROUTE_DIRECTION_NAME,SCHEDULED_TIME,ACTUAL_ARRIVAL_TIME,ADHERENCE,DWELL_IN_MINS,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,HDWY_DEV_PCT
434647,2024-08-30,Weekday,PM Peak,Summer,Y,False,3339,CH46,FROM DOWNTOWN,16:02:00,17:14:22,-451.866666,379.500000,15.0,36.716666,21.716666,144.777773
284204,2024-02-05,Weekday,PM Peak,Winter,Y,False,1997,WHBG,FROM DOWNTOWN,15:52:00,17:39:28,-114.350000,6.883333,15.0,39.416666,24.416666,162.777773
512332,2024-12-16,Weekday,Midday,Winter,N,False,3403,28&CHARL,FROM DOWNTOWN,12:56:00,14:28:01,-92.016666,0.000000,15.0,32.216666,17.216666,114.777773
512333,2024-12-16,Weekday,Midday,Winter,N,False,3403,CH46,FROM DOWNTOWN,13:01:00,14:31:49,-90.816666,0.000000,15.0,32.350000,17.350000,115.666667
512334,2024-12-16,Weekday,Midday,Winter,N,False,3403,WHBG,FROM DOWNTOWN,13:06:00,14:36:36,-90.600000,0.000000,15.0,30.683333,15.683333,104.555553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112947,2023-06-09,Weekday,PM Peak,Summer,N,False,2629,WHBG,FROM DOWNTOWN,15:07:00,15:27:02,-20.033333,0.000000,15.0,33.600000,18.600000,124.000000
472431,2024-10-22,Weekday,PM Peak,Fall,Y,False,2109,WHBG,FROM DOWNTOWN,16:23:00,16:43:02,-20.033333,0.000000,16.0,35.833333,19.833333,123.958331
456958,2024-09-30,Weekday,Midday,Fall,N,False,2832,CH46,TO DOWNTOWN,13:05:00,13:25:02,-20.033333,0.000000,15.0,36.250000,21.250000,141.666667
78009,2023-04-21,Weekday,PM Peak,Spring,N,False,1277,CH46,FROM DOWNTOWN,16:32:00,16:52:02,-20.033333,0.000000,15.0,31.150000,16.150000,107.666667


In [35]:
non_null_headway[(non_null_headway['ADHERENCE'] < -20)].sort_values('ADHERENCE')

,DATE,DAY_TYPE,TIME_OF_DAY,SEASON,IN_SCHOOL_ZONE_TIME,IS_HOLIDAY,OPERATOR,TIME_POINT_ABBR,ROUTE_DIRECTION_NAME,SCHEDULED_TIME,ACTUAL_ARRIVAL_TIME,ADHERENCE,DWELL_IN_MINS,SCHEDULED_HDWY,ACTUAL_HDWY,HDWY_DEV,HDWY_DEV_PCT
434647,2024-08-30,Weekday,PM Peak,Summer,Y,False,3339,CH46,FROM DOWNTOWN,16:02:00,17:14:22,-451.866666,379.500000,15.0,36.716666,21.716666,144.777773
583793,2025-03-25,Weekday,PM Peak,Spring,N,False,3296,WHBG,FROM DOWNTOWN,18:33:00,23:00:29,-267.483333,0.000000,15.0,29.533333,14.533333,96.888887
433464,2024-08-29,Weekday,Midday,Summer,N,False,1705,CH46,TO DOWNTOWN,13:49:00,14:05:18,-236.733333,220.433333,14.0,11.750000,-2.250000,-16.071429
582402,2025-03-24,Weekday,Midday,Spring,N,False,2094,28&CHARL,TO DOWNTOWN,09:24:00,12:24:24,-192.233333,11.833333,15.0,24.350000,9.350000,62.333333
582400,2025-03-24,Weekday,Midday,Spring,N,False,2094,WHBG,TO DOWNTOWN,09:16:00,12:15:58,-179.966666,0.000000,14.0,13.733333,-0.266667,-1.904764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456958,2024-09-30,Weekday,Midday,Fall,N,False,2832,CH46,TO DOWNTOWN,13:05:00,13:25:02,-20.033333,0.000000,15.0,36.250000,21.250000,141.666667
241254,2023-12-06,Weekday,PM Peak,Winter,N,False,3142,WHBG,FROM DOWNTOWN,17:37:00,17:56:12,-20.016666,0.816666,15.0,10.100000,-4.900000,-32.666667
16458,2023-01-24,Weekday,PM Peak,Winter,N,False,2109,WHBG,FROM DOWNTOWN,17:34:00,17:54:01,-20.016666,0.000000,12.0,31.316666,19.316666,160.972217
78010,2023-04-21,Weekday,PM Peak,Spring,N,False,1277,WHBG,FROM DOWNTOWN,16:37:00,16:57:01,-20.016666,0.000000,15.0,29.800000,14.800000,98.666667
